In [2]:
import pandas as pd

In [2]:

rna = pd.read_csv("rna_tpm.tsv", sep="\t", index_col=0) 
mirna = pd.read_csv("mirna.tsv", sep="\t", index_col=0).T
protein = pd.read_csv("protein.tsv", sep="\t", index_col=0).T

clinical = pd.read_csv("clinical.tsv", sep="\t", index_col=0)
subtypes = pd.read_csv("subtypes.csv", index_col=0)


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
print(rna)

## Wybrór konkretnego RNA --- zamienic nazwy na geny !!!!!!

In [3]:
import pandas as pd

df = rna
# przykładowo:
# df = pd.read_csv("rna.tsv", sep="\t", index_col=0)

# usuń część po kropce z ID
df.index = df.index.str.replace(r'\.\d+$', '', regex=True)


In [4]:
import mygene
mg = mygene.MyGeneInfo()

mapping = mg.querymany(df.index, scopes='ensembl.gene', fields='symbol', species='human')


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
29 input query terms found dup hits:	[('ENSG00000175711', 2), ('ENSG00000188660', 2), ('ENSG00000215156', 2), ('ENSG00000226506', 2), ('E
1303 input query terms found no hit:	['ENSG00000002079', 'ENSG00000002586.20_PAR_Y', 'ENSG00000112096', 'ENSG00000124333.16_PAR_Y', 'ENSG


In [5]:
# słownik: Ensembl → symbol
map_dict = {d['query']: d.get('symbol') for d in mapping if 'symbol' in d}

# teraz podmień indeks w dataframe
df = df.rename(index=map_dict)


In [6]:
df = df[~df.index.isna()]


In [7]:
df = df[~df.index.str.startswith('ENSG')]


In [8]:
df.to_csv("rna_mapped.csv")


### wybór genów

In [ ]:
rna_wybrane =  pd.read_csv("rna_mapped.csv")


In [ ]:
genes = [
    "ESR1", "PGR", "ERBB2", "EGFR", "KRT5", "KRT14", "KRT17", "FOXA1", "GATA3",
    "BCL2", "CDH1", "VIM", "CLDN3", "CLDN4", "CLDN7", "MKI67", "CCND1", "MYC",
    "TP53", "PIK3CA", "PTEN", "AKT1", "BRCA1", "BRCA2", "AR", "FGFR1",
    "KRT18", "KRT19", "MUC1", "EPCAM", "SNAI1", "SNAI2", "ZEB1", "ZEB2",
    "TWIST1", "TGFBR2", "CD24", "CD44"
]

print(rna_wybrane)


In [ ]:
available_rna = [p for p in genes if p in rna_wybrane['Ensembl_ID'].values]
missing_rna = [p for p in genes if p not in rna_wybrane['Ensembl_ID'].values]

print("Dostępne geny:", available_rna)
print("Brakujące geny:", missing_rna)

# FILTRUJ WIERSZE GDZIE Ensembl_ID JEST W available_rna
selected_dfmirna = rna_wybrane[rna_wybrane['Ensembl_ID'].isin(available_rna)]
selected_dfmirna.to_csv("wybrane_rna.tsv", sep="\t", index=False)


Dostępne geny: ['ESR1', 'PGR', 'ERBB2', 'EGFR', 'KRT5', 'KRT14', 'KRT17', 'FOXA1', 'GATA3', 'BCL2', 'CDH1', 'VIM', 'CLDN3', 'CLDN4', 'CLDN7', 'MKI67', 'CCND1', 'MYC', 'TP53', 'PIK3CA', 'PTEN', 'AKT1', 'BRCA1', 'BRCA2', 'AR', 'FGFR1', 'KRT18', 'KRT19', 'MUC1', 'EPCAM', 'SNAI1', 'SNAI2', 'ZEB1', 'ZEB2', 'TWIST1', 'TGFBR2', 'CD24', 'CD44']
Brakujące geny: []
Wybrano 38 wierszy z 38 genami


In [22]:
g = mg.getgene('ENSG00000000003',  field = 'symbol')# , 'symbol' ] )# fields= 'entrezgene, symbol') 

print(g)

{'_id': '7105', '_version': 1, 'symbol': 'TSPAN6'}


## Wybrór konkretnego miRNA

In [ ]:
wybrane_mirna = [
    # let-7 family (Supresja proliferacji)
    "hsa-let-7a-1",
    "hsa-let-7a-2",
    "hsa-let-7a-3",
    "hsa-let-7b",
    "hsa-let-7c",
    "hsa-let-7d",
    "hsa-let-7e",
    "hsa-let-7f-1",
    "hsa-let-7f-2",
    "hsa-let-7g",
    "hsa-let-7i",
    
    # miR-21 (Onkogenne)
    "hsa-mir-21",
    
    # miR-200 family (Regulacja EMT)
    "hsa-mir-141",
    "hsa-mir-200a",
    "hsa-mir-200b",
    "hsa-mir-200c",
    "hsa-mir-429",
    
    # miR-221/222 (Oporność na terapię hormonalną)
    "hsa-mir-221",
    "hsa-mir-222",
    
    # miR-205 (Marker luminalny, tłumi EMT)
    "hsa-mir-205",
    
    # miR-155 (Prozapalny)
    "hsa-mir-155",
    
    # miR-10b (Promocja przerzutów)
    "hsa-mir-10b",
    
    # miR-31 (Zahamowanie migracji)
    "hsa-mir-31",
    
    # miR-34a (Regulacja p53 i apoptozy)
    "hsa-mir-34a"
]


availablemirna = [p for p in wybrane_mirna if p in mirna.columns]
missingmirna = [p for p in wybrane_mirna if p not in mirna.columns]

print(availablemirna)
print(missingmirna)


selected_dfmirna = mirna[availablemirna]
selected_dfmirna.to_csv("wybrane_mirna.tsv", sep="\t")


['hsa-let-7a-1', 'hsa-let-7a-2', 'hsa-let-7a-3', 'hsa-let-7b', 'hsa-let-7c', 'hsa-let-7d', 'hsa-let-7e', 'hsa-let-7f-1', 'hsa-let-7f-2', 'hsa-let-7g', 'hsa-let-7i', 'hsa-mir-21', 'hsa-mir-141', 'hsa-mir-200a', 'hsa-mir-200b', 'hsa-mir-200c', 'hsa-mir-429', 'hsa-mir-221', 'hsa-mir-222', 'hsa-mir-205', 'hsa-mir-155', 'hsa-mir-10b', 'hsa-mir-31', 'hsa-mir-34a']
[]


## Wybrór konkretnego protein

In [35]:

proteins = [
    "ERALPHA", "PR", "HER2", "PHER2", "EGFR", "AKT_pS473", "PAKT", "P53", "PS6",
    "PTEN", "PMAPK", "MEK1", "CyclinB1", "Ki67", "BCL2", "BAX",
    "PARPCLEAVED", "p53", "BRCA1", "ECADHERIN", "NCADHERIN", "BETACATENIN", "PDL1"
]

protein.columns = protein.columns.str.replace(r"[^A-Za-z0-9]", "", regex=True).str.upper()

available = [p for p in proteins if p in protein.columns]
missing = [p for p in proteins if p not in protein.columns]


selected_df = protein[available]

selected_df.to_csv("protein_selected.tsv", sep="\t")

#print(selected_df.head())


In [18]:
#print("RNA IDs:", rna.index[:5])
#print("MiRNA IDs:", mirna.index[:5])
#print("Protein IDs:", protein.index[:5])
#print("Clinical IDs:", clinical.index[:5])
#print("Subtypes IDs:", subtypes.index[:5])


In [19]:
#common_ids = set(rna.index) & set(mirna.index) & set(protein.index)
#print("Liczba wspólnych pacjentek molekularnych:", len(common_ids))

#combined_molecular = rna.join([mirna, protein], how="inner")
#print("Rozmiar połączonych danych molekularnych:", combined_molecular.shape)

#print(combined_molecular.head())


# Wybór ze zbioru clinical

In [4]:
clinical = pd.read_csv("clinical-oczyszcz.tsv", sep="\t", index_col=0)


In [ ]:
#clinical.columns

In [6]:
selected_clinical = [
    "cases.submitter_id", "cases.case_id",
    "demographic.age_at_index", "demographic.gender",
    "demographic.race", "demographic.ethnicity",
    "demographic.vital_status", "demographic.days_to_death",
    "cases.days_to_lost_to_followup",
    "diagnoses.age_at_diagnosis", "diagnoses.primary_diagnosis",
    "diagnoses.ajcc_pathologic_stage", "diagnoses.ajcc_pathologic_t",
    "diagnoses.ajcc_pathologic_n", "diagnoses.ajcc_pathologic_m",
    "diagnoses.year_of_diagnosis", "diagnoses.laterality",
    "treatments.treatment_type", "treatments.treatment_outcome"
]


In [ ]:
clinical_av = [p for p in selected_clinical if p in clinical.columns]
miss_clinic = [p for p in selected_clinical if p not in clinical.columns]

print(clinical_av)
print(miss_clinic)


selected_clinical= clinical[clinical_av]
selected_clinical.to_csv("clinical_av.tsv", sep="\t")

# Wybór ze zbioru subtypes


In [10]:
subtypes = pd.read_csv("subtypes.csv")


In [13]:
selected_subtypes = [
    "patient",
    "Tumor.Type",
    "vital_status",
    "days_to_death",
    "days_to_last_followup",
    "age_at_initial_pathologic_diagnosis",
    "pathologic_stage",
    "Tumor_Grade",
    "BRCA_Pathology",
    "BRCA_Subtype_PAM50",
    "tobacco_smoking_history"
]


In [14]:
subtypes_av = [p for p in selected_subtypes if p in subtypes.columns]
miss_subtypes= [p for p in selected_subtypes if p not in subtypes.columns]

print(subtypes_av)
print(miss_subtypes)


selected_subtypes2= subtypes[subtypes_av]
selected_subtypes2.to_csv("subtypes_av.tsv", sep="\t")

['patient', 'Tumor.Type', 'vital_status', 'days_to_death', 'days_to_last_followup', 'age_at_initial_pathologic_diagnosis', 'pathologic_stage', 'Tumor_Grade', 'BRCA_Pathology', 'BRCA_Subtype_PAM50', 'tobacco_smoking_history']
[]


# Porównanie ID pacjentów między plikami

## miRNA + protein

In [ ]:

df_protein = pd.read_csv('protein_selected.tsv', sep='\t', index_col=0)
df_mirna = pd.read_csv('wybrane_mirna.tsv', sep='\t', index_col=0)

ids_protein = set(df_protein.index)
ids_mirna = set(df_mirna.index)


common_ids = len(ids_protein.intersection(ids_mirna))
different_ids = len(ids_protein.symmetric_difference(ids_mirna))

print(f"Liczba takich samych ID (wspólnych): {common_ids}")
print(f"Liczba różnych ID (łącznie w obu plikach): {different_ids}")


### Połączenie plików po id pacjenta

In [ ]:

df_combined = df_protein.join(df_mirna, how='inner')

print(df_combined)

df_combined.to_csv("protein_mirna-connected.tsv", sep="\t")

## miRNA + protein + RNA

In [46]:
rna_wybrane = pd.read_csv("wybrane_rna.tsv", sep="\t", index_col=0).T 
mirnaprot = pd.read_csv("protein_mirna-connected.tsv", sep="\t", index_col=0) 

In [ ]:
print(mirnaprot)


In [ ]:
print(rna_wybrane)

In [53]:
df_combined2 = mirnaprot.join(rna_wybrane, how='inner', lsuffix='_mirnaprot', rsuffix='_rna')

df_combined2.to_csv("protein_mirna_rna-connected.tsv", sep="\t")

# clinical + subtypes

In [20]:
df_clinical = pd.read_csv('clinical_av.tsv', sep='\t', index_col=0)
df_subtypes = pd.read_csv('subtypes_av.tsv', sep='\t', index_col=0)

In [ ]:
print(df_clinical)

In [ ]:
print(df_subtypes)

In [ ]:
merged_df = df_clinical.reset_index().merge(
    df_subtypes,
    how='inner',
    left_on='cases.submitter_id',
    right_on='patient'
)

#print("Liczba wspólnych pacjentów:", merged_df['patient'].nunique())

Liczba wspólnych pacjentów: 1086


In [24]:
merged_df.to_csv('merged_clinical_subtypes.tsv', sep='\t', index=False)


# Clinical + subtypes + protein + miRNA + RNA

In [31]:
df_geny = pd.read_csv('protein_mirna_rna-connected2.tsv', sep='\t', index_col=0)
df_ogolne = pd.read_csv('merged_clinical_subtypes.tsv', sep='\t', index_col=0)

In [32]:
df_geny.index = df_geny.index.str.replace(r'-\d+[A-Z]+$', '', regex=True)

In [33]:
df_geny.columns

Index(['ID.1', 'ERALPHA', 'PR', 'HER2', 'EGFR_mirnaprot', 'P53',
       'PTEN_mirnaprot', 'MEK1', 'BCL2_mirnaprot', 'BAX', 'PARPCLEAVED',
       'ECADHERIN', 'NCADHERIN', 'BETACATENIN', 'PDL1', 'hsa-let-7a-1',
       'hsa-let-7a-2', 'hsa-let-7a-3', 'hsa-let-7b', 'hsa-let-7c',
       'hsa-let-7d', 'hsa-let-7e', 'hsa-let-7f-1', 'hsa-let-7f-2',
       'hsa-let-7g', 'hsa-let-7i', 'hsa-mir-21', 'hsa-mir-141', 'hsa-mir-200a',
       'hsa-mir-200b', 'hsa-mir-200c', 'hsa-mir-429', 'hsa-mir-221',
       'hsa-mir-222', 'hsa-mir-205', 'hsa-mir-155', 'hsa-mir-10b',
       'hsa-mir-31', 'hsa-mir-34a', 'BRCA1', 'SNAI2', 'VIM', 'CD44', 'CDH1',
       'FGFR1', 'PGR', 'ESR1', 'GATA3', 'CCND1', 'KRT18', 'EPCAM', 'PIK3CA',
       'TWIST1', 'SNAI1', 'KRT17', 'FOXA1', 'MYC', 'BRCA2', 'TP53', 'ERBB2',
       'AKT1', 'EGFR_rna', 'ZEB1', 'MKI67', 'TGFBR2', 'CLDN3', 'AR', 'ZEB2',
       'KRT19', 'BCL2_rna', 'PTEN_rna', 'CLDN7', 'MUC1', 'KRT5', 'KRT14',
       'CLDN4', 'CD24'],
      dtype='object')

In [34]:
merged_df2 = df_geny.reset_index().merge(
    df_ogolne,
    how='inner',
    left_on='ID.1',
    right_on='cases.submitter_id'
)

print("Liczba wspólnych pacjentów:", merged_df2['cases.submitter_id'].nunique())

Liczba wspólnych pacjentów: 695


In [35]:
merged_df2.to_csv('merged_data.tsv', sep='\t', index=False)
